In [1]:
import sqlite3
import pandas as pd
from pathlib import Path
import re
import sqlite3
import json
import random

In [2]:
# connect to db and get a verse from the main db
def connect_to_dbsqlite3():
    path_db = Path.cwd().parent / "eu_verses_db.sqlite"
    connect = sqlite3.connect(path_db)
    cur = connect.cursor()
    return connect, cur

connect, cur = connect_to_dbsqlite3()

quiry_result = cur.execute("SELECT * FROM raw_verses")
print(quiry_result.fetchone())

(1, 'avva.html', '2012-01-16', 'Авва', '\nАвва,\nзачем высыхает небо?\nПроступает соль?\nИ каждому, иногда совсем не в меру,\nбросают в открытую рану веры\nщепотку - для сохранения веры боль?\n\nАвва,\nкак рождается забота Ребекки\nдля тех, кто не может поднять веки,\nиспечь пару окраин хлеба\nс тем же горчайшим привкусом неба -\nВсе той же потери боль?\n\nОткуда у сына берутся силы\nсплести из бедренной жилы\nкротчайшего нрава\nгулко звучащее слово -\n"Авва"?\n\nСкажи мне,\nчтобы утихла боль...\n')


In [3]:
#read sql raw_verses to pd
pd_verses = pd.read_sql_query("SELECT * FROM raw_verses",connect)

# combine all verses into one string 
def get_verses_as_str(pd_verses):
    verses_as_str = str()
    for item in pd_verses["verses"]:
        verses_as_str += item
    return verses_as_str

verses_as_str = get_verses_as_str(pd_verses)

# remove punctuation to leave onle unique separate words
verses_as_separate_words = re.sub(r"[^а-я|A-Я|ё|Ё]", ' ', verses_as_str).split()
verses_as_unique_words = list(set(verses_as_separate_words))

# make list [id, ("word")]
def make_id_word(pd_verses, verses_as_unique_words):
    list_id_word = list()
    for word in verses_as_unique_words:
        id = pd_verses[pd_verses["verses"].str.contains(word)]["id"].iloc[0]
        id = int(id)
        list_id_word.append((id, word.lower()))
    return list_id_word


list_id_word = make_id_word(pd_verses, verses_as_unique_words)
print("number of words id",len(list_id_word))
print("example of id words",list_id_word[0:3])
connect.close()

number of words id 1290
example of id words [(16, 'младенца'), (18, 'грядущий'), (21, 'прогулку')]


In [16]:
# create table id_word and populate it with id = number of verse
# word = unique word, this verse contains
connect, cur = connect_to_dbsqlite3()
quiry_0 = """
DROP TABLE IF EXISTS id_word;
    CREATE TABLE IF NOT EXISTS id_word (
        id_word PRIMARY KEY,
        id_raw_verses INT, 
        word VARCHAR(255), 
        FOREIGN KEY(id_raw_verses) REFERENCES raw_versese(id)
    );
"""
quiry_1 = "INSERT INTO id_word VALUES('id_raw_verses', 'word')"
quiry_2 = "SELECT * FROM id_word"

quiry_result = cur.execute(quiry_2)

cur.executescript(quiry_0)
connect.commit()

cur.executemany(quiry_1, list_id_word)
connect.commit()

quiry_result = cur.execute(quiry_2)
print(quiry_result.fetchone())

OperationalError: table id_word has 3 columns but 2 values were supplied

In [12]:
# create dict_id_json_words
def create_list_id_json_words(list_id_word):
    n_unique_id = list(set((id_word[0] for id_word in list_id_word)))
    list_id_json_words = []
    for i in n_unique_id:
        list_words = []
        for id_word in list_id_word:
            id, word = id_word[0], id_word[1]
            if id == i:
                list_words.append(word)
        json_words = json.dumps(list_words)
        list_id_json_words.append(tuple((i, json_words)))
    return list_id_json_words

list_id_json_words = create_list_id_json_words(list_id_word)
print(list_id_json_words[10][0])
        

11


In [13]:
# write id_json_words to sqlite
quiry_3 = """
    DROP TABLE IF EXISTS id_json_words;
    CREATE TABLE IF NOT EXISTS id_json_words (
        id_json INT,
        words VARCHAR(65535), 
        FOREIGN KEY (id_json) REFERENCES raw_verses(id) 
        );
    """
cur.executescript(quiry_3)
cur.executemany("INSERT INTO id_json_words VALUES (?, ?)", 
                [(item[0], item[1]) for item in list_id_json_words]
                ) 
connect.commit()
example = cur.execute("SELECT words FROM id_json_words WHERE id_json=1;").fetchall()
print(json.loads(example[0][0]))

['веры', 'во', 'мне', 'для', 'ребекки', 'сына', 'на', 'открытую', 'о', 'может', 'под', 'все', 'ре', 'привкусом', 'чтоб', 'рану', 'сплести', 'и', 'в', 'все', 'хлеба', 'веки', 'с', 'утихла', 'сохранения', 'в', 'дня', 'испечь', 'каждому', 'совсем', 'проступает', 'тем', 'потери', 'силы', 'б', 'кто', 'по', 'чем', 'авва', 'забота', 'его', 'чтобы', 'пару', 'той', 'окраин', 'щепотку', 'зачем', 'ране', 'бедренной', 'до', 'хра', 'н', 'рождается', 'звучащее', 'соль', 'откуда', 'высыхает', 'что', 'всем', 'ни', 'горчайшим', 'неба', 'дом', 'у', 'как', 'поднять', 'я', 'берутся', 'из', 'вкус', 'за', 'гулко', 'к', 'от', 'но', 'ос', 'же', 'не', 'слово', 'иногда', 'а', 'бросают', 'то', 'кротчайшего', 'и', 'жилы', 'скажи', 'нрава', 'о', 'а', 'тех', 'бы', 'меру', 'небо', 'боль']


In [14]:
# check tables created in sqlite
print("raw_verses", cur.execute("SELECT COUNT(*) FROM raw_verses").fetchall())
print("id_word",cur.execute("SELECT COUNT(*) FROM id_word").fetchall())
print("id_json_words", cur.execute("SELECT COUNT(*) FROM id_json_words").fetchall())
connect.close()

raw_verses [(22,)]
id_word [(1290,)]
id_json_words [(22,)]


In [15]:
# get dict {random word: verse id} drom db.sqlite3 
connect, cur = connect_to_dbsqlite3()
def get_random_words():
    all_ids = cur.execute("SELECT id_json FROM id_json_words").fetchall()
    dict_random_words_id = {} 
    for i_d in all_ids:
        words = cur.execute("SELECT words FROM id_json_words WHERE id_json=?", i_d).fetchall() 
        words = json.loads(words[0][0])
        random_word = words[random.randint(0, len(words)-1)]
        dict_random_words_id[random_word] = i_d[0]
    return dict_random_words_id

dict_random_words_id = get_random_words()
print("len of random words =", len(dict_random_words_id))  
print(dict_random_words_id) 
connect.close()

len of random words = 22
{'о': 1, 'пения': 2, 'подножие': 3, 'твердыней': 4, 'тяжести': 5, 'пахнет': 6, 'троллейбусные': 7, 'неровен': 8, 'изъять': 9, 'погрешность': 10, 'голос': 11, 'выходит': 12, 'замирает': 13, 'мак': 14, 'коснется': 15, 'отпечатки': 16, 'казнить': 17, 'день': 18, 'поленья': 19, 'дверь': 20, 'весне': 21, 'гладом': 22}
